In [1]:
sys.path.append("..")
import os
import utils
import numpy as np
import pandas as pd

In [2]:
def read_all_results(dir_path):
    paths = os.listdir(dir_path)
    try: paths.remove('archive') 
    except: pass
    last_gens = []
    for i in range(len(paths)):
        path = dir_path + paths[i]
        last_gen = sorted(map(lambda x: x.split("_"), 
                        os.listdir(f"{path}/individuals/")), 
                        key=lambda x: int(x[1].split(".")[0]))[-1][-1][:-4]
        print(f"Path: {paths[i]}, last generation: {int(last_gen)}")
        last_gens.append(int(last_gen))
    return paths, last_gens

In [3]:
dir_path = "../results/"
paths, last_gens = read_all_results(dir_path)

Path: GA_2021_05_16_23_21_10, last generation: 43
Path: GA_2021_05_16_23_22_48, last generation: 43
Path: GA_2021_05_16_23_19_30, last generation: 43
Path: GA_2021_05_16_23_20_22, last generation: 43
Path: GA_2021_05_16_23_21_41, last generation: 15
Path: GA_2021_05_16_23_19_57, last generation: 45
Path: GA_2021_05_16_23_22_10, last generation: 43
Path: GA_2021_05_16_23_20_39, last generation: 48


In [27]:
run = 7
gen = 48
path = dir_path + paths[run]

individuals_path =  f"{path}/individuals/individuals_{gen}.pkl"
score_path       =  f"{path}/final_scores/final_score_{gen}.pkl"
overview_path    =  f"{path}/generation_overview.csv"

final_score = utils.read_pickle(score_path)
individuals = utils.read_pickle(individuals_path)
overview = pd.read_csv(overview_path)

In [28]:
with open(dir_path + paths[run] + "/run_params.json", "r") as file:
    print(file.read())
    file.close()

{
    "objective": "yll",
    "simulations": 20,
    "process": {
        "horizon": 74,
        "decision_period": 28,
        "policy": "commuter_based"
    },
    "population_size": 40,
    "random_individuals": true,
    "min_generations": 40
}


In [29]:
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n//10%10!=1)*(n%10<4)*n%10::4])
for ix, i in enumerate(sorted(individuals, key=lambda x: x.mean_score)):
    if ix == 0:
        print("Strategy count:")
        print(f"{'U':>5}{'D':>5}{'N':>5}")
        for w in range(1,4):
            print(f"{w}: {i.strategy_count[gen]['U'][w]:<5}{i.strategy_count[gen]['D'][w]:<5}{i.strategy_count[gen]['N'][w]:<5}")
        print()
    print(f"{utils.tcolors.BOLD}{ordinal(ix+1)} place:{utils.tcolors.ENDC}")
    print(f"ID: {i.ID}")
    print(f"Score: {np.round(i.mean_score, 3)}")
    
    print(f"Genes:\n{np.round(i.genes, 3)}")
    print()

Strategy count:
    U    D    N
1: 16   20   11   
2: 5    9    2    
3: 0    1    0    

1st place:
ID: gen_26_008
Score: 39887.25
Genes:
[[[0.028 0.13  0.466 0.022 0.065 0.29 ]
  [0.457 0.    0.    0.125 0.    0.418]
  [0.    0.    0.994 0.    0.    0.006]]

 [[0.    0.    0.834 0.    0.    0.166]
  [0.233 0.242 0.312 0.    0.029 0.184]
  [0.013 0.    0.861 0.033 0.046 0.047]]

 [[0.145 0.438 0.277 0.    0.    0.14 ]
  [0.254 0.087 0.086 0.254 0.005 0.315]
  [0.019 0.157 0.202 0.041 0.57  0.011]]]

2nd place:
ID: gen_18_013
Score: 39888.25
Genes:
[[[0.032 0.129 0.464 0.022 0.064 0.289]
  [0.585 0.    0.    0.    0.    0.415]
  [0.    0.    1.    0.    0.    0.   ]]

 [[0.    0.    0.668 0.    0.    0.332]
  [0.241 0.242 0.322 0.    0.03  0.166]
  [0.014 0.    0.901 0.035 0.    0.05 ]]

 [[0.145 0.438 0.277 0.    0.    0.14 ]
  [0.255 0.088 0.086 0.255 0.    0.317]
  [0.018 0.155 0.201 0.041 0.585 0.   ]]]

3rd place:
ID: gen_39_009
Score: 39892.7
Genes:
[[[0.031 0.129 0.464 0.022 0.0

In [30]:
print(individuals[0].genes, end="\n"*2)
genes = np.sum(individuals[0].genes, axis=(0,1))
norm = np.sum(genes)
norm_genes = np.divide(genes, norm)
print(norm_genes)

[[[0.02772154 0.12981317 0.46594438 0.02156354 0.06452118 0.2904362 ]
  [0.45667618 0.         0.         0.12511083 0.         0.41821299]
  [0.         0.         0.99358962 0.         0.         0.00641038]]

 [[0.         0.         0.83418633 0.         0.         0.16581367]
  [0.23311384 0.2417248  0.31167746 0.         0.02920702 0.18427689]
  [0.01268598 0.         0.86100509 0.03305378 0.04611612 0.04713902]]

 [[0.14493941 0.43808816 0.27745665 0.         0.         0.13951578]
  [0.25355091 0.08738514 0.08570492 0.25355091 0.0047617  0.31504641]
  [0.01869097 0.15650874 0.20236493 0.0408563  0.5704409  0.01113814]]]

[0.12748654 0.11705778 0.44799215 0.05268171 0.07944966 0.17533217]
